In [2]:
import pandas as pd
import numpy as np
movies = pd.read_csv('movies_dataset.csv', low_memory=False)
movies.shape

(45466, 24)

### 1) Eliminar las columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_path y homepage.

In [3]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
movies.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'],inplace=True)
movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [5]:
movies.shape

(45466, 18)

### 2) Los valores nulos del campo release date deben eliminarse.

### 3) De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [6]:
movies['release_date'] = pd.to_datetime(movies['release_date'], format = "%Y-%m-%d", errors= 'coerce')

# se usa el metodo "to_datetime" para convertir a fecha, se le indica el formato requerido, y aquellos registros
# que no puedan ser convertidos, generando error, son convertidos a nulo.

In [7]:
movies.dropna(subset=['release_date'], inplace=True)

# se usa "dropna" con "inplace=True" para lo eliminar los registros requeridos, en este caso se usa
# "subset=['release_date']" para que sean solo las filas que tienen nulos en esta columna.

In [8]:
# teniendo todos los valores de fecha validos, creamos la columna con solo el año de lanzamiento

movies['release_year'] = movies['release_date'].dt.year

In [ ]:
print(movies.shape)

movies[['release_date','release_year']]
# comprobando que se creo release_year correctamente

(45376, 19)


,release_date,release_year
0,1995-10-30,1995
1,1995-12-15,1995
2,1995-12-22,1995
3,1995-12-22,1995
4,1995-02-10,1995
...,...,...
45460,1991-05-13,1991
45462,2011-11-17,2011
45463,2003-08-01,2003
45464,1917-10-21,1917


### 4) Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [10]:
# corroborando el tipo de dato de las columnas que seran usadas
movies['revenue'].dtype

dtype('float64')

In [11]:
movies['budget'].dtype

dtype('O')

In [12]:
# para calcular el return mas adelante, debemos cambiar el typo de dato en "budget" a float
movies['budget']= movies['budget'].astype(float)
movies['budget'].dtype

dtype('float64')

In [ ]:
movies['revenue'].notna().value_counts()
# no hay valores nulos en revenue

revenue
True    45376
Name: count, dtype: int64

In [ ]:
movies['budget'].notna().value_counts()

# no hay valores nulos en budget

budget
True    45376
Name: count, dtype: int64

### 5) Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [15]:
# buscando si hay valores negativos en budget

movies[movies['budget'] < 0].shape

(0, 19)

In [16]:
# buscando si hay valores negativos en revenue

movies[movies['revenue'] < 0].shape

(0, 19)

In [17]:
movies['return'] = movies['revenue']/movies['budget']
movies['return']

0        12.451801
1         4.043035
2              NaN
3         5.090760
4              inf
           ...    
45460          NaN
45462          NaN
45463          NaN
45464          NaN
45465          NaN
Name: return, Length: 45376, dtype: float64

In [18]:
movies['return'].isna().value_counts()

return
True     34464
False    10912
Name: count, dtype: int64

In [19]:
# hay 2 tipos de errores aca, 0/0 que dara nulo, y n/0 (budget = 0) que dara infinito
# ambos valores seran reemplazados por 0

movies['return'].replace([np.inf, np.nan], 0, inplace=True)

In [ ]:
movies['return'].isna().value_counts()
# comprobando que no hay valores nulos

return
False    45376
Name: count, dtype: int64

### 6) Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

# La desanidacion de estos campos (ultimo item requerido) se realizó en el notebook "ETL_modelo" y "ETL_credits"